In [ ]:
import time
import threading
from os import getcwd
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import UnexpectedAlertPresentException, StaleElementReferenceException, NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException, ElementNotInteractableException, NoSuchElementException
from selenium.common.exceptions import TimeoutException, InvalidSessionIdException
import logging
from tenacity import *
from dataclasses import dataclass

from re import sub
from decimal import Decimal

In [ ]:
current_dir = getcwd()
prefs = {'download.default_directory' : f'{current_dir}/Downloads',
        'credentials_enable_service': False, 'profile.password_manager_enabled' : False}
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--log-level=3")
chrome_options.add_argument("--start-maximized")
chrome_options.add_experimental_option('prefs', prefs)
chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])
chrome_options.add_experimental_option("excludeSwitches",["enable-automation"])
chrome_options.add_experimental_option("useAutomationExtension", False)

In [ ]:

website = 'https://www.pichau.com.br/hardware/placa-de-video'
s=Service(ChromeDriverManager(log_level=logging.CRITICAL,path=current_dir).install())
driver = webdriver.Chrome(service=s,options=chrome_options, service_log_path='NUL')
driver.get(website)

In [ ]:
running = False
processedCards = []

In [ ]:
@dataclass
class Placa():
    nome:str
    preco:Decimal
    link:str

In [ ]:
def searchPichau():

    global processedCards, running

    if (EC.any_of(EC.visibility_of_element_located((By.XPATH, '//*[@id="rcc-confirm-button"]')))(driver)):
        driver.find_element(By.XPATH,'//*[@id="rcc-confirm-button"]').click()

    WebDriverWait(driver, 5).until(EC.invisibility_of_element_located((By.XPATH, '//*[@id="rcc-confirm-button"]')))

    orderBox = By.ID, 'demo-customized-select'
    loading = By.CSS_SELECTOR, '#__next > main > div.MuiBackdrop-root.dark-mode'
    driver.find_element(*orderBox).click()
    pathMenor = By.XPATH, '//*[@id="menu-"]/div[3]/ul/li[4]'

    WebDriverWait(driver, 5).until(EC.visibility_of_element_located(pathMenor)).click()
    time.sleep(2)
    WebDriverWait(driver, 10).until(EC.all_of(EC.text_to_be_present_in_element(orderBox, 'Menor valor'),EC.invisibility_of_element_located(loading)))
    running = True
    
    selPages = By.XPATH, '//*[@id="__next"]/main/div[2]/div/div[1]/nav/ul'
    selNext = By.XPATH, 'li[9]/button'
    
    while running:
        searchPage()
        print('Analisando nova pagina...')
        pages = driver.find_element(*selPages)
        action = ActionChains(driver)
        nextBtn = pages.find_element(*selNext)
        action.move_to_element(nextBtn).perform()
        time.sleep(1)
        nextBtn.click()
        time.sleep(5)

    print(f'Consulta finalizada!. {len(processedCards)} placas processadas.')

In [ ]:
def searchPage():
    global processedCards, running

    placas = [a for a in driver.find_elements(By.CSS_SELECTOR, "[data-cy*='list-product']")]

    for placa in placas:
        if running:
            details = placa.find_element(By.CLASS_NAME, 'MuiCardContent-root')
            if EC.all_of(EC.visibility_of_element_located((By.CLASS_NAME,'jss191')))(details):
                running = False
                break

            link = placa.get_attribute('href')
            nome = details.find_element(By.TAG_NAME, 'h2').text
            preco = details.find_element(By.CLASS_NAME, 'jss200').text.replace('.','').replace(',','.')
            
            preco = Decimal(sub(r'[^\d.]', '', preco))
            placa = Placa(link=link,nome=nome,preco=preco)
            processedCards.append(placa)

In [ ]:
searchPichau()

In [ ]:
[a for a in processedCards if a.preco>2000 and a.preco<5000]